# Module 3: Transfer Learning with ResNet50 (Google Colab Version)

This notebook implements the transfer learning pipeline. It is designed to run on Google Colab, mounting your Google Drive to access the dataset.

In [2]:
import tensorflow as tf
import sys

print(f"Python Version: {sys.version}")
print(f"TensorFlow Version: {tf.__version__}")

# Check for GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\n✅ GPU Detected: {len(gpus)} found")
    for gpu in gpus:
        print(f" - Name: {gpu.name}")
        print(f" - Type: {gpu.device_type}")
        
    # If on Colab/Linux with NVIDIA GPU, this gives more details:
    try:
        print("\n--- NVIDIA-SMI Output ---")
        !nvidia-smi
    except:
        print("nvidia-smi not found (Expected if on Mac or TPU)")
else:
    print("\n❌ No GPU Detected. Running on CPU.")

Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
TensorFlow Version: 2.19.0

✅ GPU Detected: 1 found
 - Name: /physical_device:GPU:0
 - Type: GPU

--- NVIDIA-SMI Output ---
Sat Dec  6 17:27:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P8              9W /   70W

In [1]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Drive Mounted!")

ValueError: mount failed

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ValueError: mount failed

In [ ]:
# 2. Configuration
import os

# UPDATE THIS PATH to point to your dataset folder in Google Drive
# Example: "/content/drive/MyDrive/Projects/FakeCurrency/Final_Clean_Dataset"
DRIVE_DATASET_PATH = "/content/drive/MyDrive/Final_Clean_Dataset"

# We copy the dataset to the local Colab VM for speed (reading from Drive is slow)
LOCAL_DATASET_PATH = "/content/Final_Clean_Dataset"

if not os.path.exists(LOCAL_DATASET_PATH):
    print("Copying dataset to local runtime (this speeds up training)...")
    # If it's a folder, we use cp -r. If it's a zip, we unzip.
    # Assuming it is a folder based on your description.
    !cp -r "$DRIVE_DATASET_PATH" "$LOCAL_DATASET_PATH"
    print("Copy complete.")
else:
    print("Dataset already exists locally.")

DATASET_ROOT = LOCAL_DATASET_PATH

In [ ]:
# 3. Imports & Setup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

OUTPUT_DIR = "/content/Module3_Results"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001

In [ ]:
# 4. Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

print(f"Loading data from: {DATASET_ROOT}")

train_generator = train_datagen.flow_from_directory(
    DATASET_ROOT,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = val_datagen.flow_from_directory(
    DATASET_ROOT,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

In [ ]:
# 5. Build & Train Model
def build_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()

callbacks = [
    ModelCheckpoint(os.path.join(OUTPUT_DIR, 'model_resnet50_best.h5'), 
                    monitor='val_loss', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
# 6. Evaluation
# Plot History
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()

# Confusion Matrix
validation_generator.reset()
preds = model.predict(validation_generator, verbose=1)
y_pred = (preds > 0.5).astype(int).flatten()
y_true = validation_generator.classes

print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()